Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from scipy.stats import uniform
from sklearn.decomposition import TruncatedSVD
import spacy


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [4]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [5]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
3106,4245,"\nNew distillery in Lancaster, Penn. “Locally sourced” grain, copper pot still, unaged. Rich new-make smell, warm, full grain, rye oil, and anise. Light spirit: sweet, more grain and a character like pizzelles; sweet crisp anise cookies. Easily enjoyed, with a finish of more sweet grain and anise. Interestingly schnapps-like, given the area’s German roots.",0
3128,4962,"\nAn odd roaring noseful of spices: vanilla, pepper, ginger, teaberry. Tastes perfumey first, then resolves to sweet vanilla, hot oak, and more teaberry. Can’t decide if it wants to be bourbon—it’s hot, and flexing oak—or candy, and does a poor job at both; it’s like a linebacker in a French maid’s outfit, just wrong. There are much better sweet, spicy whiskey mixes out there; the original Spicebox comes to mind.",0
314,4310,"\nLemon-lime gum drops and fresh-cut grass notes, with an underlying creamy, marshmallow-tinged maltiness. Plenty of vanilla, along with a hint of ginger spice zing. Clean, nicely rounded, and fully mature for its age.",0


In [6]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
1849,5037,"\nThis is new make from the Rittenhouse Rye mashbill, unaged, right off the still. Yeasty mash notes come through strongly, with a big nose of rye and corn underneath. It smells fresh, and alive. Wow, that’s powerful stuff. Much hotter than the Trybox Corn, this one is on fire with rye spice; it’s got me breaking a sweat! A bit of water brings out more grain notes, and some tempering sweetness, but it’s still no pussycat. Cleanly powerful.",2
625,5059,"\nNasty smell of chicken coop: dried guano, some ammonia, and dust, with grain underneath. Happily, it tastes quite a bit better. Sweet grain, snap of rye spice, some dry cocoa powder, but there's still a dustiness to it, and a distinct small-barrel woodiness. Hard to get past that aroma. Seems like there's more than one problem here.",2
774,5103,"\nThis flavored moonshine smells like a mass of wet, used tea bags. Underneath is some faint lemon, honey, and black pepper, but it’s hard to get to it through the tea. Entry is strong, oversteeped tea, which only gets stronger, more tannic, and more bitter in the mid-palate. Syrupy lemon honey tries and fails to balance this oversteeped mess. Finish is long, bitter, slightly hot, and unpleasant. Do not attempt to drink without copious amounts of crushed ice.",2


### Split the Training Set into Train/Validation

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(3269,) (3269,) (818,) (818,)


In [8]:
from sklearn.pipeline import Pipeline

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2)) #vectorizer
clf = LinearSVC() #classifier

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [9]:
parameters = {
    'vect__max_df': (0.3, .5),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 20000),
    'clf__penalty': ('l1','l2'),
    'clf__C': (0.1, 0.5, 1., 2.)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  1.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [10]:
grid_search.best_score_

0.7575254449309149

In [11]:
grid_search.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__vect', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__norm', 'estimator__vect__preprocessor', 'estimator__vect__smooth_idf', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__sublinear_tf', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__use_idf', 'estimator__vect__vocabulary', 'estimator__clf__C', 'estimator__clf__class_weight', 'estimator__clf__dual', 'estimator__clf__fit_intercept', 'estimator__clf__intercept_scaling', 'estimator__clf__loss', 'estimator__clf__max_iter', 'estimator__clf__multi_class', 'estimato

In [12]:
params = grid_search.best_params_

In [13]:
params

{'clf__C': 0.5,
 'clf__penalty': 'l2',
 'vect__max_df': 0.5,
 'vect__max_features': 20000,
 'vect__min_df': 2}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [14]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [15]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [16]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [17]:
subNumber = 0

In [18]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


In [19]:
logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,random_state=0)
distributions = dict(C=uniform(loc=0, scale=4),penalty=['l2', 'l1'])
RSCV = RandomizedSearchCV(logistic, distributions, random_state=0)

In [20]:
RSCV.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__C', 'estimator__class_weight', 'estimator__dual', 'estimator__fit_intercept', 'estimator__intercept_scaling', 'estimator__l1_ratio', 'estimator__max_iter', 'estimator__multi_class', 'estimator__n_jobs', 'estimator__penalty', 'estimator__random_state', 'estimator__solver', 'estimator__tol', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'iid', 'n_iter', 'n_jobs', 'param_distributions', 'pre_dispatch', 'random_state', 'refit', 'return_train_score', 'scoring', 'verbose'])

### Define Pipeline Components

In [21]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
svd = TruncatedSVD(algorithm='randomized', n_iter=10) 
rfc = RandomForestClassifier(n_jobs = -1)

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', rfc)         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [22]:
g_params = {
    'clf__max_depth': (10,42,None),
    'clf__n_estimators': (11,21,31,41,51),
    'clf__max_leaf_nodes':(10,20,30,40,60,80,None),
    'vect__max_df': (0.3, .5),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 20000),
}

In [23]:
GSCV_search = GridSearchCV(pipe, g_params, cv=5, n_jobs=-1, verbose=1)
GSCV_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 1260 candidates, totalling 6300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 17.1min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 20.8min
[Parallel(n_jobs=-1)]: Done 6300 out of 6300 | elapsed: 21.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [24]:
GSCV_search.best_score_

0.7154378383554839

In [26]:
r_params = {
    'clf__max_depth': (10,42,None),
    'clf__n_estimators': (11,21,31,41,51),
    'clf__max_leaf_nodes':(10,20,30,40,60,80,None),
    'vect__max_df': (0.3, .5),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 20000),
}

In [27]:
rando_search = RandomizedSearchCV(pipe, r_params, cv=5, n_jobs=-1, verbose=1, random_state = 17)
rando_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   11.3s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

In [28]:
rando_search.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__vect', 'estimator__svd', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__norm', 'estimator__vect__preprocessor', 'estimator__vect__smooth_idf', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__sublinear_tf', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__use_idf', 'estimator__vect__vocabulary', 'estimator__svd__algorithm', 'estimator__svd__n_components', 'estimator__svd__n_iter', 'estimator__svd__random_state', 'estimator__svd__tol', 'estimator__clf__bootstrap', 'estimator__clf__ccp_alpha', 'estimator__clf__clas

In [29]:
rando_search.best_params_

{'vect__min_df': 2,
 'vect__max_features': 5000,
 'vect__max_df': 0.3,
 'clf__n_estimators': 31,
 'clf__max_leaf_nodes': 80,
 'clf__max_depth': 10}

In [30]:
rando_search.best_score_

0.7105445709001565

### Make a Submission File

In [31]:
# Predictions on test sample
pred = rando_search.predict(test['description'])

In [32]:
# Predictions on test sample
pred = rando_search.predict(test['description'])

In [33]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [34]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [35]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

In [36]:
# Predictions on test sample
pred = GSCV_search.predict(test['description'])

In [37]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [38]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [39]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [40]:
# Apply to your Dataset
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint
import scipy.stats as stats
# Use parameters from GridSearch in previous section
vect = TfidfVectorizer(parameters)

# Add dimensionality reduction
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

# Use parameters from RandomSearch in previous section
clf = LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=None,
                           tol=0.0001, verbose=0)
param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [41]:
nlp = spacy.load("en_core_web_lg")


In [42]:
train['emb'] = train['description'].apply(lambda text: nlp(text))

In [43]:
test['emb'] = test['description'].apply(lambda text: nlp(text))

In [ ]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2)) #vectorizer
clf = LinearSVC() #classifier

pipe = Pipeline([('vect', vect), ('clf', clf)])
parameters = {
    'vect__max_df': (0.3, .5),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 20000),
    'clf__penalty': ('l1','l2'),
    'clf__C': (0.1, 0.5, 1., 2.)
}
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['emb'], train['ratingCategory'])

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
C:\Python\Anaconda3\envs\U4-S1-NLP\lib\site-packages\joblib\externals\loky\process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt7/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?